In [38]:
import networkx as nx

import pandas as pd
import numpy as np

from bokeh.io import show, output_file, export_png, push_notebook,output_notebook
from bokeh.models import CDSView,Legend,Slider,BooleanFilter,IndexFilter,CustomJSFilter,CustomJS, LegendItem, ResetTool, Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, SaveTool, BoxZoomTool, WheelZoomTool, PanTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import figure,ColumnDataSource
from bokeh.core.properties import field

from bokeh.layouts import widgetbox,row
#from bokeh.models.widgets import Slider

from networkx.drawing.nx_agraph import pygraphviz_layout
#import matplotlib.pyplot as plt

from argparse import ArgumentParser
import pickle
#output_notebook()

In [39]:
samplecol=['#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#800000', '#aaffc3', '#808000', '#000075', '#808080', '#000000']

In [40]:
pathtabla='v2_edges_06_after_cleaning_checkcontrol_dereplication.csv'
pathcolores='colores_bgc_clado.csv'
uniform='red'
nxlayout=''
progname='neato'
ecolor='#CCCCCC'
ewidth=1
ndia=8
bgcgroup='NRPS'
salida='v2_color_clado_checkcontrol_dereplication'
origen='clado_categories_v2.csv'

In [41]:
a=pd.read_csv(pathtabla)

info=pd.read_csv('v2_nodes_after_cleaning_checkcontrol_dereplication.csv')
infoex=info.copy()
infoex1=infoex[['Id','reduccion']]

g1=nx.from_pandas_edgelist(a, source='Source', target='Target', edge_attr=['Raw distance'])

xg=pd.DataFrame({'Id':[i for i in nx.nodes(g1)]})
xgredu=pd.merge(xg,infoex1,how='left',on='Id')
nx.set_node_attributes(g1, dict(zip(xgredu['Id'],xgredu['reduccion'])),'Class')
if bgcgroup!='':
    tempo=g1.copy()
    g2=tempo.subgraph([x for x,y in tempo.nodes(data=True) if y['Class']==bgcgroup])
    g1=g2.copy()
    faltantes=list(set(info[info['reduccion']=='NRPS']['Id'])-set(list(nx.nodes(g1))))
    g1.add_nodes_from(faltantes,Class='NRPS')
    lg=pd.DataFrame({'Id':[i for i in nx.nodes(g1)]})
else:
    lg=xg

In [42]:
f1=lambda x:x.split('.')[0]+'.1'

In [43]:
with open('rhodo_contig_genome_dict.pkl','rb') as pr1:
    dictr1=pickle.load(pr1)

In [44]:
with open('rhodo_assembly_name.pkl','rb') as pr2:
    dictr2=pickle.load(pr2)

In [45]:
back=[dictr2[dictr1[y]] for y in list(map(f1,faltantes))]
len(pd.unique(back))

19

In [46]:
#info=pd.read_csv('nodes_after_cleaning_dereplication.csv')
lg['contigId']=lg['Id'].apply(lambda x: x.split('.')[0]+'.1')

In [47]:
if origen!='':
    yi=pd.read_csv(origen,sep=';')
    yi.iloc[:,0]=yi.iloc[:,0].apply(lambda x:x[0:-8])
    dict1=dict(zip(yi.iloc[:,0],yi.iloc[:,1]))
with open('rhodo_contig_genome_dict.pkl','rb') as f2: 
    dict2=pickle.load(f2)
with open('rhodo_assembly_name.pkl','rb') as f3: 
    dict3=pickle.load(f3)

lg['Clade']=lg['contigId'].apply(lambda x:dict1[dict2[x]])
#lg2['cat']=lg2['contigId'].apply(lambda x:dict1[dict2[x]])
lg['bacteria']=lg['contigId'].apply(lambda x:dict3[dict2[x]])

q=pd.read_csv('v2_gcf_after_cleaning_checkcontrol_dereplication.csv')
lg2=lg.merge(q,how='left',on='Id')
lg2.loc[lg2['gcf'].isna(),'gcf']='NA'

In [48]:
lg2[lg2['bacteria'].apply(lambda x: 'B7g' in x)]

,Id,contigId,Clade,bacteria,gcf
48,NZ_LQWU01000001.cluster022,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,8
65,NZ_LQWU01000001.cluster023,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,22
150,NZ_LQWU01000001.cluster012,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,6
208,NZ_LQWU01000001.cluster013,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,2
351,NZ_LQWU01000001.cluster014,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,10
407,NZ_LQWU01000001.cluster009,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,16
422,NZ_LQWU01000001.cluster004,NZ_LQWU01000001.1,4b,Rhodococcus erythropolis strain B7g,3


In [49]:
if pathcolores!='':
    tabcol=pd.read_csv(pathcolores,header=None)
    colores=dict(zip(tabcol.iloc[:,0],tabcol.iloc[:,1]))
    colli=[colores[i] if (j!="Rhodococcus sp. H-CA8f") else 'white' for i,j in zip(lg2['Clade'],lg2['bacteria'])]
    #colores={q:samplecol[cou%len(samplecol)] for cou,q in zip(range(len(pd.unique(lg2['gcf']))),pd.unique(lg2['gcf']))}
    #colli=[colores[i] if (j not in ["Rhodococcus sp. H-CA8f"]) else 'white' for i,j in zip(lg2['gcf'],lg2['bacteria'])] #,"Rhodococcus erythropolis strain B7g"
else:
    colli=[uniform for i in lg2['Clade']]

In [50]:
colores

{'1': '#F2cd6f',
 '2a': '#00721f',
 '2b': '#8db955',
 '3a': '#d6c6e1',
 '3b': '#781c45',
 '4a': '#7db6ff',
 '4b': '#2f5ccc'}

In [51]:
nx.set_node_attributes(g1, dict(zip(lg2['Id'],lg2['Clade'])),'Clade')
nx.set_node_attributes(g1, dict(zip(lg2['Id'],colli)),'ncolor')
nx.set_node_attributes(g1, dict(zip(lg2['Id'],lg2['bacteria'])),'Bacteria')
nx.set_node_attributes(g1, dict(zip(lg2['Id'],lg2['gcf'])),'GCF')
nx.set_node_attributes(g1, dict(zip(lg2['Id'],lg2['Id'])),'Cluster')

gy=pygraphviz_layout(g1,prog=progname)

with open('layout_neato_NRPS_checkcontrol_dereplication_v2.pkl','wb') as f1:
    pickle.dump(gy,f1)

In [52]:
with open('layout_neato_NRPS_checkcontrol_dereplication_v2.pkl','rb') as f:
    gy=pickle.load(f)

In [53]:
len(nx.nodes(g1))

717

In [54]:
if nxlayout!='':
    plot = Plot(plot_width=1000, plot_height=1000,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
    result = getattr(nx,nxlayout+'_layout')

    graph_renderer = from_networkx(g1,result, scale=1, center=(0,0))
else:
    xmaxplot=max([h[0] for h in gy.values()])+20
    ymaxplot=max([h[1] for h in gy.values()])+20
    xminplot=min([h[0] for h in gy.values()])-20
    yminplot=min([h[1] for h in gy.values()])-20

    plot = Plot(plot_width=1000, plot_height=1000,
            x_range=Range1d(xminplot,xmaxplot), y_range=Range1d(yminplot,ymaxplot))

    graph_renderer = from_networkx(g1,gy,prog=progname)#scale=1, center=(0,0)

plot.add_tools(HoverTool(tooltips=[('Bacteria','@Bacteria'),('Cluster','@Cluster'),('Class','@Class'),('Clade','@Clade'),('GCF','@GCF')]), ResetTool(), TapTool(), BoxSelectTool(),SaveTool(),WheelZoomTool(), PanTool(),BoxZoomTool())

graph_renderer.node_renderer.glyph = Circle(size=ndia, fill_color={'field':'ncolor'})
graph_renderer.edge_renderer.glyph = MultiLine(line_color=ecolor, line_alpha=1, line_width=ewidth)
graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

output_file(salida+".html")

show(plot)

In [ ]:
js_filter = CustomJSFilter(args=dict(slider=slider, source=source), code=code)

In [ ]:
code = '''
var N = slider.value
var indices = [];
for (var i = 0; i <= source.data['Raw distance'].length; i++){
    if (source.data['Raw distance'][i] <= N) {
        indices.push(true)
    }
    else{
        indices.push(false)
    }
}
return indices;
'''